# 0.0. Imports

In [4]:
#Import Libraries
import pickle
import inflection

import numpy as np
import pandas as pd
import seaborn as sn

from ydata_profiling import ProfileReport

from matplotlib import pyplot as plt

from xgboost import XGBClassifier

from sklearn import svm
from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp
from sklearn import linear_model as lm
from sklearn import ensemble as en

# 0.1. Functions

In [5]:
#Rename dataframe columns
def rename_cols(df):
    cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
                'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
                'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
                'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 
                'Reserva_Cancelada', 'latitude', 'longitude']
    
    snakecase = lambda x: inflection.underscore(x)
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    
    return df

def rename_cols_x(df):
    cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
                'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
                'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
                'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 'latitude', 'longitude']
    
    snakecase = lambda x: inflection.underscore(x)
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    
    return df

#Suiting data to algorithm application
def feature_eng(df):
    df['classificacao_do_hotel'] = df['classificacao_do_hotel'].apply(lambda x: x.replace(' estrelas', ''))
    
    #Suiting data to algorithm application
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Sim', '1'))
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Não', '0'))
    
    #Suiting data to algorithm application
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Sim', '1'))
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Não', '0'))
    
    #Suiting data to algorithm application
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].apply(lambda x: x.replace('Sim', '1'))
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].apply(lambda x: x.replace('Não', '0'))
    
    #Suiting data to algorithm application
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].apply(lambda x: x.replace('Sim', '1'))
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].apply(lambda x: x.replace('Não', '0'))
    
    #Adjusting data types
    df['classificacao_do_hotel'] = df['classificacao_do_hotel'].astype('int64')
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].astype('int64')
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].astype('int64')
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].astype('int64')
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].astype('int64')
    
    return df

#Optimizing values for algorithm application
def feature_opt(df):
    #Make a frequency dictionary of each category
    freq_dict = df['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['tipo_do_quarto_reservado_encoded'] = df['tipo_do_quarto_reservado'].map(freq_dict)
    
    #Make a frequency dictionary of each category
    freq_dict = df['nacionalidade'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['nacionalidade'] = df['nacionalidade'].map(freq_dict)
    
    #Dummy variables
    cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']  
    df_dummy = pd.get_dummies(df[cols_dummy])  
    df = pd.concat([df, df_dummy], axis=1)
    
    #Adjusting values for algorithm application
    df['latitude'] = df['latitude'].str.replace('.', '')
    df['longitude'] = df['longitude'].str.replace('.', '')
    df['latitude'] = df['latitude'].astype('float64')
    df['longitude'] = df['longitude'].astype('float64')
    
    #Apply radians to data for optimize
    df['latitude'] = df['latitude'].apply(lambda x: np.radians(x/1000000))
    df['longitude'] = df['longitude'].apply(lambda x: np.radians(x/1000000))
    
    return df

def feature_opt_x(df):
    #Make a frequency dictionary of each category
    freq_dict = df['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['tipo_do_quarto_reservado_encoded'] = df['tipo_do_quarto_reservado'].map(freq_dict)
    
    #Make a frequency dictionary of each category
    freq_dict = df['nacionalidade'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['nacionalidade'] = df['nacionalidade'].map(freq_dict)
    
    #Dummy variables
    cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']  
    df_dummy = pd.get_dummies(df[cols_dummy])  
    df = pd.concat([df, df_dummy], axis=1)
    
    #Adjusting values for algorithm application
    df['latitude'] = df['latitude'].str.replace('.', '')
    df['longitude'] = df['longitude'].str.replace('.', '')
    df['latitude'] = df['latitude'].astype('float64')
    df['longitude'] = df['longitude'].astype('float64')
    
    #Apply radians to data for optimize
    df['latitude'] = df['latitude'].fillna(40463667)
    df['longitude'] = df['longitude'].fillna(-374922)
    df['latitude'] = df['latitude'].apply(lambda x: np.radians(x/1000000))
    df['longitude'] = df['longitude'].apply(lambda x: np.radians(x/1000000))
    
    return df


#Adjusting dataframe types
def types_adjust(df):
    df['numero_de_hospedes'] = df['numero_de_hospedes'].astype('int64')
    df['classificacao_do_hotel'] = df['classificacao_do_hotel'].astype('int64')
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].astype('int64')
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].astype('int64')
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].astype('int64')
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].astype('int64')
    
    return df

# 1.0. Data Description

## 1.1. Loading Data

In [6]:
#Import train data and country data (for merge)
df_train = pd.read_csv('data/train.csv')
df_country = pd.read_csv('data/country_base.csv', delimiter=';')

#Merge train and country data
df1 = pd.merge(df_train, df_country, on='Nacionalidade', how='left')

#Drop dataframe column
df1 = df1.drop('country', axis=1)

In [7]:
#Import test data and country data (for merge)
df_test = pd.read_csv('data/test.csv')
df_country = pd.read_csv('data/country_base.csv', delimiter=';')

#Merge train and country data
df1_x = pd.merge(df_test, df_country, on='Nacionalidade', how='left')

#Drop dataframe column
df1_x = df1_x.drop('country', axis=1)

## 1.2. Rename Columns

In [8]:
#Applying Rename Columns Function
df1 = rename_cols(df1)
df1_x = rename_cols_x(df1_x)

## 1.3. Data Dimensions

In [9]:
#Counting number of rows and columns of the dataframe
print('Train Number of rows: {}'.format(df1.shape[0]))
print('Train Number of cols: {}'.format(df1.shape[1]))
print('\n')
print('Test Number of rows: {}'.format(df1_x.shape[0]))
print('Test Number of cols: {}'.format(df1_x.shape[1]))

Train Number of rows: 72159
Train Number of cols: 17


Test Number of rows: 48106
Test Number of cols: 16


## 1.4. Check NA

In [10]:
#Check of NA's values os de dataframe
df1.isna().sum()

id                                         0
classificacao_do_hotel                     0
meses_da_reserva_ate_o_check_in            0
numero_de_pernoites_reservadas             0
numero_de_hospedes                         3
regime_de_alimentacao                      0
nacionalidade                           1093
forma_de_reserva                           0
ja_se_hospedou_anterioremente              0
tipo_do_quarto_reservado                   0
reserva_feita_por_agencia_de_turismo       0
reserva_feita_por_empresa                  0
reserva_com_estacionamento                 0
reserva_com_observacoes                    0
reserva_cancelada                          0
latitude                                1093
longitude                               1093
dtype: int64

In [11]:
df1_x.isna().sum()

id                                        0
classificacao_do_hotel                    0
meses_da_reserva_ate_o_check_in           0
numero_de_pernoites_reservadas            0
numero_de_hospedes                        1
regime_de_alimentacao                     0
nacionalidade                           677
forma_de_reserva                          0
ja_se_hospedou_anterioremente             0
tipo_do_quarto_reservado                  0
reserva_feita_por_agencia_de_turismo      0
reserva_feita_por_empresa                 0
reserva_com_estacionamento                0
reserva_com_observacoes                   0
latitude                                677
longitude                               677
dtype: int64

### 1.4.1. Drop / Replace NA's

In [12]:
#Drop NA's from train dataframe
df1 = df1.dropna()

#Replace NA's from test dataframe
df1_x['numero_de_hospedes'] = df1_x['numero_de_hospedes'].fillna(2)
df1_x['nacionalidade'] = df1_x['nacionalidade'].fillna('Spain')
df1_x['latitude'] = df1_x['latitude'].fillna(40463667)
df1_x['longitude'] = df1_x['longitude'].fillna(-374922)

## 1.5. Data Dtypes

In [13]:
#Checking train data dtypes
df1.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
latitude                                 object
longitude                                object
dtype: object

In [14]:
#Checking test data dtypes
df1_x.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
latitude                                 object
longitude                                object
dtype: object

## 1.6. Data Balancing

In [15]:
#Checking test data balance
df1['reserva_cancelada'].value_counts(normalize=True)

0    0.62674
1    0.37326
Name: reserva_cancelada, dtype: float64

## 1.7. Pandas Profiling

In [16]:
#Report for External Data Analysis
prof = ProfileReport(df1)
prof.to_file(output_file='Profile_Report.html')
# regime_de_alimentacao -> desbalanceado
# nacionalidade -> desbalanceado
# forma_de_reserva -> desbalanceado
# ja_se_hospedou_anterioremente -> desbalanceado
# tipo_do_quarto_reservado -> desbalanceado
# reserva_feita_por_agencia_de_turismo -> desbalanceado
# reserva_com_estacionamento -> desbalanceado

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# 2.0. Feature Engineering

In [17]:
#Dataframe copy for data adjustment
df2 = df1.copy()
df2_x = df1_x.copy()

In [18]:
#Applying Feature Engineering Function
df2 = feature_eng(df2)
df2_x = feature_eng(df2_x)

In [19]:
#Checking data adjustments
df2

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
0,33571,5,5,5,2.0,Café da manha e jantar,France,Agência,0,Amethyst,1,0,1,Nenhuma,0,46.227.638,2.213.749
1,82458,4,167,3,2.0,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40.463.667,-374.922
2,94061,4,4,3,2.0,Café da manha,Belgium,Agência,0,Amethyst,1,0,0,1 a 3,0,50.503.887,4.469.936
3,75196,4,13,2,2.0,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40.463.667,-374.922
4,82940,4,4,2,2.0,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40.463.667,-374.922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72154,101851,4,2,3,2.0,Café da manha,France,Agência,0,Green Emerald,1,0,0,1 a 3,0,46.227.638,2.213.749
72155,50800,4,4,2,2.0,Sem refeicao,Italy,Agência,0,Amethyst,1,0,1,1 a 3,0,4.187.194,1.256.738
72156,4101,5,5,2,2.0,Café da manha e jantar,Spain,Agência,0,Amethyst,0,0,0,Nenhuma,1,40.463.667,-374.922
72157,15828,5,1,8,2.0,Café da manha e jantar,Spain,Balcão,0,Pink Sapphire,1,0,0,Nenhuma,0,40.463.667,-374.922


In [20]:
#Checking data types
df2.dtypes

id                                        int64
classificacao_do_hotel                    int64
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente             int64
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo      int64
reserva_feita_por_empresa                 int64
reserva_com_estacionamento                int64
reserva_com_observacoes                  object
reserva_cancelada                         int64
latitude                                 object
longitude                                object
dtype: object

In [21]:
df2_x

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,latitude,longitude
0,118345,4,8,13,3.0,Café da manha,South Korea,Balcão,0,Green Emerald,1,0,0,Nenhuma,35.907.757,127.766.922
1,9500,5,5,4,769.0,Café da manha,United States,Agência,0,Pink Sapphire,1,0,0,Nenhuma,3.709.024,-95.712.891
2,34558,5,4,11,1.0,Café da manha e jantar,United Kingdom,Agência,0,Amethyst,0,0,0,Nenhuma,55.378.051,-3.435.973
3,70816,4,7,4,2.0,Café da manha,Spain,Agência,0,Amethyst,0,0,0,Nenhuma,40.463.667,-374.922
4,105321,4,1,8,2.0,Café da manha,Italy,B2B,0,Amethyst,0,1,0,1 a 3,4.187.194,1.256.738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48101,23143,5,1,7,2.0,Café da manha e jantar,United Kingdom,Agência,0,Green Emerald,1,0,0,Nenhuma,55.378.051,-3.435.973
48102,78403,4,1,4,1.0,Café da manha,Belgium,Agência,0,Amethyst,1,0,0,Nenhuma,50.503.887,4.469.936
48103,95111,4,7,1,4.0,Café da manha,Germany,Agência,0,Tanzanite,1,0,0,1 a 3,51.165.691,10.451.526
48104,68717,4,4,3,1.0,Café da manha,Spain,Agência,0,Amethyst,0,0,0,Nenhuma,40.463.667,-374.922


In [22]:
df2_x.dtypes

id                                        int64
classificacao_do_hotel                    int64
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente             int64
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo      int64
reserva_feita_por_empresa                 int64
reserva_com_estacionamento                int64
reserva_com_observacoes                  object
latitude                                 object
longitude                                object
dtype: object

# 3.0. Data Filtering

In [23]:
#This section was not used for this project

# 4.0. Data Preparation 

In [24]:
#Dataframe copy for data preparation
df3 = df2.copy()
df3_x = df2_x.copy()

In [25]:
#Applying Feature Optimizer Function
df3 = feature_opt(df3)

df3_x = feature_opt_x(df3_x)

C:\Users\Caio\AppData\Local\Temp\ipykernel_14152\462635688.py:76: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['latitude'] = df['latitude'].str.replace('.', '')
C:\Users\Caio\AppData\Local\Temp\ipykernel_14152\462635688.py:77: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['longitude'] = df['longitude'].str.replace('.', '')
C:\Users\Caio\AppData\Local\Temp\ipykernel_14152\462635688.py:106: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['latitude'] = df['latitude'].str.replace('.', '')
C:\Users\Caio\AppData\Local\

In [26]:
#Checking train data types
df3.dtypes

id                                                        int64
classificacao_do_hotel                                    int64
meses_da_reserva_ate_o_check_in                           int64
numero_de_pernoites_reservadas                            int64
numero_de_hospedes                                      float64
regime_de_alimentacao                                    object
nacionalidade                                           float64
forma_de_reserva                                         object
ja_se_hospedou_anterioremente                             int64
tipo_do_quarto_reservado                                 object
reserva_feita_por_agencia_de_turismo                      int64
reserva_feita_por_empresa                                 int64
reserva_com_estacionamento                                int64
reserva_com_observacoes                                  object
reserva_cancelada                                         int64
latitude                                

In [27]:
#Checking test data types
df3_x.dtypes

id                                                        int64
classificacao_do_hotel                                    int64
meses_da_reserva_ate_o_check_in                           int64
numero_de_pernoites_reservadas                            int64
numero_de_hospedes                                      float64
regime_de_alimentacao                                    object
nacionalidade                                           float64
forma_de_reserva                                         object
ja_se_hospedou_anterioremente                             int64
tipo_do_quarto_reservado                                 object
reserva_feita_por_agencia_de_turismo                      int64
reserva_feita_por_empresa                                 int64
reserva_com_estacionamento                                int64
reserva_com_observacoes                                  object
latitude                                                float64
longitude                               

In [28]:
#Checking train data optimized
df3[['tipo_do_quarto_reservado', 'tipo_do_quarto_reservado_encoded', 'nacionalidade', 'latitude', 'longitude']]

,tipo_do_quarto_reservado,tipo_do_quarto_reservado_encoded,nacionalidade,latitude,longitude
0,Amethyst,0.718447,0.087964,0.806824,0.038637
1,Amethyst,0.718447,0.481390,0.706224,-0.006544
2,Amethyst,0.718447,0.020601,0.881459,0.078015
3,Amethyst,0.718447,0.481390,0.706224,-0.006544
4,Amethyst,0.718447,0.481390,0.706224,-0.006544
...,...,...,...,...,...
72154,Green Emerald,0.163517,0.087964,0.806824,0.038637
72155,Amethyst,0.718447,0.031381,0.073080,0.021934
72156,Amethyst,0.718447,0.481390,0.706224,-0.006544
72157,Pink Sapphire,0.053628,0.481390,0.706224,-0.006544


In [29]:
#Checking test data optimized
df3_x[['tipo_do_quarto_reservado', 'tipo_do_quarto_reservado_encoded', 'nacionalidade', 'latitude', 'longitude']]

,tipo_do_quarto_reservado,tipo_do_quarto_reservado_encoded,nacionalidade,latitude,longitude
0,Green Emerald,0.157964,0.000873,0.626709,2.229953
1,Pink Sapphire,0.054920,0.018023,0.064735,-1.670505
2,Amethyst,0.723008,0.101068,0.966529,-0.059969
3,Amethyst,0.723008,0.491061,0.706224,-0.006544
4,Amethyst,0.723008,0.031929,0.073080,0.021934
...,...,...,...,...,...
48101,Green Emerald,0.157964,0.101068,0.966529,-0.059969
48102,Amethyst,0.723008,0.018251,0.881459,0.078015
48103,Tanzanite,0.023760,0.060159,0.893010,0.182414
48104,Amethyst,0.723008,0.491061,0.706224,-0.006544


# 5.0. Feature Selection 

In [30]:
#Dataframe copy for feature selection
df4 = df3.copy()
df4_x = df3_x.copy()

In [31]:
#Applying data types adjustment functions
df4 = types_adjust(df4)
df4_x = types_adjust(df4_x)

In [32]:
#Checking train data
df4.dtypes

id                                                        int64
classificacao_do_hotel                                    int64
meses_da_reserva_ate_o_check_in                           int64
numero_de_pernoites_reservadas                            int64
numero_de_hospedes                                        int64
regime_de_alimentacao                                    object
nacionalidade                                           float64
forma_de_reserva                                         object
ja_se_hospedou_anterioremente                             int64
tipo_do_quarto_reservado                                 object
reserva_feita_por_agencia_de_turismo                      int64
reserva_feita_por_empresa                                 int64
reserva_com_estacionamento                                int64
reserva_com_observacoes                                  object
reserva_cancelada                                         int64
latitude                                

In [33]:
#Checking test data
df4_x.dtypes

id                                                        int64
classificacao_do_hotel                                    int64
meses_da_reserva_ate_o_check_in                           int64
numero_de_pernoites_reservadas                            int64
numero_de_hospedes                                        int64
regime_de_alimentacao                                    object
nacionalidade                                           float64
forma_de_reserva                                         object
ja_se_hospedou_anterioremente                             int64
tipo_do_quarto_reservado                                 object
reserva_feita_por_agencia_de_turismo                      int64
reserva_feita_por_empresa                                 int64
reserva_com_estacionamento                                int64
reserva_com_observacoes                                  object
latitude                                                float64
longitude                               

In [34]:
#Drop columns from dataframe
cols_drop =['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes', 'tipo_do_quarto_reservado']
df5 = df4.drop(cols_drop, axis=1)
df5_x = df4_x.drop(cols_drop, axis=1)

In [35]:
#Checking train data types for Machine Learning Application
df5.dtypes

id                                                        int64
classificacao_do_hotel                                    int64
meses_da_reserva_ate_o_check_in                           int64
numero_de_pernoites_reservadas                            int64
numero_de_hospedes                                        int64
nacionalidade                                           float64
ja_se_hospedou_anterioremente                             int64
reserva_feita_por_agencia_de_turismo                      int64
reserva_feita_por_empresa                                 int64
reserva_com_estacionamento                                int64
reserva_cancelada                                         int64
latitude                                                float64
longitude                                               float64
tipo_do_quarto_reservado_encoded                        float64
regime_de_alimentacao_Café da manha                       uint8
regime_de_alimentacao_Café da manha e ja

In [36]:
#Checking test data types for Machine Learning Application
df5_x.dtypes

id                                                        int64
classificacao_do_hotel                                    int64
meses_da_reserva_ate_o_check_in                           int64
numero_de_pernoites_reservadas                            int64
numero_de_hospedes                                        int64
nacionalidade                                           float64
ja_se_hospedou_anterioremente                             int64
reserva_feita_por_agencia_de_turismo                      int64
reserva_feita_por_empresa                                 int64
reserva_com_estacionamento                                int64
latitude                                                float64
longitude                                               float64
tipo_do_quarto_reservado_encoded                        float64
regime_de_alimentacao_Café da manha                       uint8
regime_de_alimentacao_Café da manha e jantar              uint8
regime_de_alimentacao_Café da manha, alm

In [37]:
#Split train dataset into features and target
#X = df5.drop(['id', 'reserva_cancelada'], axis=1)
X = df5.drop(['reserva_cancelada'], axis=1)
y = df5['reserva_cancelada']

In [38]:
#Split train dataset into train and test
X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2)

# 6.0. Machine Learning Model

## 6.1. XGBoost

In [39]:
#Model Definition
xgb_model = XGBClassifier(n_estimators=1000, max_depth=20, min_child_weight=1)

#Model Training
xgb_model.fit(X_train, y_train.values.ravel())

#Model Prediction
yhat_xgb = xgb_model.predict(X_val)

In [40]:
#Export the model pkl for test data application
#pickle.dump(xgb_model, open('xgb_model.pkl', 'wb' ))

### 6.1.1. Performance

In [41]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_xgb)
recall = mt.recall_score(y_val, yhat_xgb)
f1_score = mt.f1_score(y_val, yhat_xgb, average='macro')

In [42]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96665
Recall: 0.95970
F1 Score: 0.97074


## 6.2. Gradient Boosting

In [43]:
#Model Definition
gdr_model = en.GradientBoostingClassifier(n_estimators=500, random_state=42, max_depth=15)

#Model Training
gdr_model.fit(X_train, y_train)

#Model Prediction
yhat_gb = gdr_model.predict(X_val)

In [44]:
#Export the model pkl for test data application
#pickle.dump(gdr_model, open('gdr_model.pkl', 'wb' ))

### 6.2.1. Performance

In [45]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_gb)
recall = mt.recall_score(y_val, yhat_gb)
f1_score = mt.f1_score(y_val, yhat_gb, average='macro')

In [46]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96589
Recall: 0.95894
F1 Score: 0.97013


## 6.3. Random Forest

In [47]:
#Model Definition
rf_model = en.RandomForestClassifier(n_estimators=1000,
                                     criterion='gini', 
                                     random_state=42, 
                                     n_jobs=-1, 
                                     max_depth=25)

#Model Training
rf_model.fit(X_train, y_train)

#Model Prediction
yhat_rf = rf_model.predict(X_val)

In [48]:
#Export the model pkl for test data application
#pickle.dump(rf_model, open('rf_model.pkl', 'wb' ))

### 6.3.1. Performance

In [49]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_rf)
recall = mt.recall_score(y_val, yhat_rf)
f1_score = mt.f1_score(y_val, yhat_rf, average='macro')

In [50]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96229
Recall: 0.96083
F1 Score: 0.96942


# 7.0. Ensemble

In [51]:
#Merge data to compare
ensemble = pd.DataFrame([yhat_xgb, yhat_gb, yhat_rf]).T
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']
moda = ensemble.mode(axis=1)[0]
ensemble['pred'] = moda

# 8.0. General Performance

In [52]:
#Collect values for test performance
yhat_en = ensemble['pred']

In [53]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_en)
recall = mt.recall_score(y_val, yhat_en)
f1_score = mt.f1_score(y_val, yhat_en, average='macro')

In [54]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96774
Recall: 0.95913
F1 Score: 0.97095


# 9.0. Model Application to Test Data

In [55]:
#XGBoost application to test data
yhat_xbg_x = xgb_model.predict(df5_x)

#Gradient Boosting application to test data
yhat_gb_x = gdr_model.predict(df5_x)

#Random Forest application to test data
yhat_rf_x = rf_model.predict(df5_x)

# 10.0. Export CSV

In [56]:
#Merge data to compare
ensemble = pd.DataFrame([yhat_xbg_x, yhat_gb_x, yhat_rf_x]).T
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']
moda_x = ensemble.mode(axis=1)[0]
ensemble['pred'] = moda_x

In [57]:
#Prepare data to export
yhat_final = ensemble['pred']
df2_x['Reserva Cancelada'] = yhat_final
submission = df2_x[['id', 'Reserva Cancelada']].copy()

In [58]:
#Export data
submission.to_csv('data/submission.csv', index=False)